In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import vgg19
from tensorflow.keras import backend as K
import cv2

# ***CONTENT LOSS***

$
    \mathcal{L}_{\text{content}} = \frac{1}{2}  \sum_{i, j} (F_{ij}^{\text{generated}} - F_{ij}^{\text{original}})^2
$

In [ ]:
def content_loss(content, generated):
    return 1/2*K.sum(K.square(generated - content))

# ***STYLE LOSS WITH GRAM MATRIX***

$
G_{ij}^l = \sum_{k} (F_{ik}^l F_{jk}^l)
$

In [ ]:
def gram_matrix(layer):
    features = K.batch_flatten(K.permute_dimensions(layer,(2,0,1)))
    gram = K.dot(features, K.transpose(features))
    return gram

$
E_{\text{style}} = \frac{1}{4 \text{N}_{\text{l}}^2 \text{M}_{\text{l}}^2} \sum_{i, j} (G_{\text{ij}}^l-S_{\text{ij}}^l)^2
$

In [ ]:
def style_loss(style, generated):
    style_gram = gram_matrix(style)
    generated_gram = gram_matrix(generated)
    N = K.cast(K.shape(style)[-1], tf.float32)
    M = K.cast(K.shape(style)[1]*K.shape(style)[2], tf.float32)
    return K.sum(K.square(generated_gram - style_gram)) / (4.0 * (N**2) * (M**2))

$
\mathcal{L}_{\text{style}} = \sum_{l=0}^{L} w_l E_l
$

In [ ]:
def total_style_loss(style_layers):
    total_style_loss = K.variable(0.0)
    for style_layer in style_layers:
        style_feature = outputs_dict[style_layer][0, :, :, :]
        generated_feature = outputs_dict[style_layer][2, :, :, :]
        loss_style = style_loss(style_feature, generated_feature)
        total_style_loss += loss_style / len(style_layers)
    return total_style_loss

# ***STYLE LOSS WITH ADAIN-BASED***

In [ ]:
def mean_std(x):
    mean = K.mean(x, axis=[0, 1, 2], keepdims=True)
    std = K.std(x, axis=[0, 1, 2], keepdims=True)
    return mean, std

def style_loss_adain(style, combination):
    mean_s, std_s = mean_std(style)
    mean_c, std_c = mean_std(combination)
    return K.sum(K.square(mean_c - mean_s)) + K.sum(K.square(std_c - std_s))


# ***TOTAL VARIATION LOSS***

$
\mathcal{L}_{tv} = \sum ((x_{i,j} - x_{i+1, j})^2 + (x_{i,j} - x_{i, j+1})^2)^{1.25}
$

In [ ]:
def total_variation_loss(image):
    x = K.square(image[:, :-1, :-1, :] - image[:, 1:, :-1, :])
    y = K.square(image[:, :-1, :-1, :] - image[:, :-1, 1:, :])
    return K.sum(K.pow(x + y, 1.25))

# ***TOTAL LOSS***

$
\mathcal{L}_{total} = \alpha \mathcal{L}_{content} + \beta \mathcal{L}_{style} + \gamma \mathcal{L}_{tv}
$

In [ ]:
def total_loss(content_loss, style_loss, total_variation_loss, content, style, image, alpha=1.0, beta=0.01, gamma=0.1):
    return alpha * content_loss(content, image) + beta * style_loss(style, image) + gamma * total_variation_loss(image)

# ***EVALUATE***

In [ ]:
content = cv2.imread(r"C:\Users\ADMIN\OneDrive\Desktop\CS406\Lab3_23520655\Mirai.jpg")
generated = cv2.imread(r"C:\Users\ADMIN\OneDrive\Desktop\CS406\Lab3_23520655\Mirai.jpg")
# content và generated được tiền xử lý từ trước


model  = vgg19.VGG19(weights='imagenet', include_top=False)
outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])

content_layer = model.get_layer('block4_conv2').output
feature_layers_style = [
    'block1_conv1',
    'block2_conv1',
    'block3_conv1',
    'block4_conv1',
    'block5_conv1'
]

feature_extractor = tf.keras.Model(inputs=model.input, outputs=content_layer)
original_features = feature_extractor(content)
generated_features = feature_extractor(generated)

In [ ]:
content_loss_value = content_loss(original_features, generated_features)
style_loss_value = total_style_loss(feature_layers_style)
total_variation_loss_value = total_variation_loss(generated)
total_loss_value = total_loss(content_loss_value, style_loss_value, total_variation_loss_value)